In [1]:
import json
import pandas as pd
import importlib
import pickle

In [2]:
filepath = '../scraping/articles/filtered_articles.json'
with open(filepath, 'r') as file:
    articles = pd.read_json(filepath)

filepath = '../relation_extraction/directory.pkl'

with open(filepath, 'rb') as file:
    keywords = pickle.load(file)

In [6]:
import KnowledgeExtractor
importlib.reload(KnowledgeExtractor)

extractor = KnowledgeExtractor.KnowledgeExtractor('dslim/bert-base-NER', keywords)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
length = 10
for i in range(10):
    sents = articles.iloc[i]['text']
    entities = extractor.extract_entities(sents)
    for e in entities:
        print(e)
    break

{'s_id': 0, 'word': 'Bishkek', 'entity': 'ORG', 'start': 0, 'end': 7}
{'s_id': 1, 'word': 'Kyrgyzstan', 'entity': 'LOC', 'start': 0, 'end': 10}
{'s_id': 1, 'word': 'Kyrgyz', 'entity': 'LOC', 'start': 253, 'end': 259}
{'s_id': 1, 'word': 'nuclear power plant', 'entity': 'REACTOR', 'start': 59, 'end': 78}
{'s_id': 3, 'word': 'Sadyr Japarov', 'entity': 'PER', 'start': 19, 'end': 32}
{'s_id': 4, 'word': 'Issyk-Kul Province', 'entity': 'LOC', 'start': 50, 'end': 68}
{'s_id': 4, 'word': 'Kyzyl-Ompol', 'entity': 'LOC', 'start': 92, 'end': 103}
{'s_id': 4, 'word': 'Japarov', 'entity': 'PER', 'start': 190, 'end': 197}
{'s_id': 5, 'word': 'Kyrgyzstan', 'entity': 'LOC', 'start': 32, 'end': 42}
{'s_id': 5, 'word': 'Russia', 'entity': 'LOC', 'start': 135, 'end': 141}
{'s_id': 5, 'word': 'Rosatom', 'entity': 'ORG', 'start': 176, 'end': 183}
{'s_id': 5, 'word': 'Asian', 'entity': 'MISC', 'start': 210, 'end': 215}
{'s_id': 5, 'word': 'small modular reactor', 'entity': 'SMR', 'start': 89, 'end': 110}
{

In [9]:
from relik import Relik
from relik.inference.data.objects import RelikOutput

relik = Relik.from_pretrained("sapienzanlp/relik-relation-extraction-nyt-large")
relik_out: RelikOutput = relik("Michael Jordan was one of the best players in the NBA.")


                ___              __         
               /\_ \      __    /\ \        
 _ __     __   \//\ \    /\_\   \ \ \/'\    
/\`'__\ /'__`\   \ \ \   \/\ \   \ \ , <    
\ \ \/ /\  __/    \_\ \_  \ \ \   \ \ \\`\  
 \ \_\ \ \____\   /\____\  \ \_\   \ \_\ \_\
  \/_/  \/____/   \/____/   \/_/    \/_/\/_/
                                            
                                            

[2024-08-06 10:35:56,018] [INFO] [relik.inference.annotator.from_pretrained:694] [PID:29125] [RANK:0] Loading Relik from sapienzanlp/relik-relation-extraction-nyt-large
[2024-08-06 10:35:56,020] [INFO] [relik.inference.annotator.from_pretrained:695] [PID:29125] [RANK:0] {
    '_target_': 'relik.inference.annotator.Relik',
    'index': {
        'triplet': {
            '_target_': 'relik.retriever.indexers.inmemory.InMemoryDocumentIndex.from_pretrained',
            'name_or_path': 'sapienzanlp/relik-retriever-small-nyt-document-index',
        },
    },
    'metadata_fields': [],
    'r

In [29]:
text = 'Central Asia served as an important source of uranium in the former Soviet Union. According to the European Bank for Reconstruction and Development (EBRD), large amount of radioactively contaminated material was placed in mining waste dumps and tailing sites. '
relik_out: RelikOutput = relik(text)

[2024-08-06 10:51:39,501] [DEBUG] [relik.reader.data.relik_reader_re_data.__iter__:399] [PID:29125] [RANK:0] Dataset finished: 3 number of elements processed


In [34]:
from pprint import pprint
from dataclasses import asdict

relik_out_dict = asdict(relik_out)
triplets = relik_out_dict['triplets']

def structure_relik_triplets(triplets):
    structured_triplets = []
    for triplet in triplets:
        print(triplet)
        subject, label, obj, confidence = triplet.subject, triplet.label, triplet.object, triplet.confidence
        structured_triplets.append({
            "head": subject.text,
            "label": label,
            "tail": obj.text,
            "confidence": confidence
        })

    return structured_triplets

structure_relik_triplets(triplets)

Triplets(subject=Span(start=8, end=12, label='--NME--', text='Asia'), label='contains', object=Span(start=68, end=80, label='--NME--', text='Soviet Union'), confidence=1.0)
Triplets(subject=Span(start=149, end=153, label='--NME--', text='EBRD'), label='company', object=Span(start=99, end=147, label='--NME--', text='European Bank for Reconstruction and Development'), confidence=1.0)
Triplets(subject=Span(start=8, end=12, label='--NME--', text='Asia'), label='contains', object=Span(start=68, end=80, label='--NME--', text='Soviet Union'), confidence=1.0)
Triplets(subject=Span(start=149, end=153, label='--NME--', text='EBRD'), label='company', object=Span(start=99, end=147, label='--NME--', text='European Bank for Reconstruction and Development'), confidence=1.0)


[{'head': 'Asia',
  'label': 'contains',
  'tail': 'Soviet Union',
  'confidence': 1.0},
 {'head': 'EBRD',
  'label': 'company',
  'tail': 'European Bank for Reconstruction and Development',
  'confidence': 1.0}]

In [15]:
sents = articles.iloc[0]['text']
for s in sents:
    print(s)

Bishkek says it is in serious need of ‘alternative sources of revenue’
Kyrgyzstan, which is considering construction of its first nuclear power plant, has lifted a ban on uranium extraction to boost the economy with lawmakers overturning a 2019 law banning the research, exploration and development of uranium reserves, the Kyrgyz parliament said in a statement. 
The government has said it is lifting the ban because of a “serious need for alternative sources of revenue”. 
Populist president Sadyr Japarov has long called for state-led production in the extractive industries, a policy that he first championed more than a decade ago as an opposition politician.
Addressing locals earlier this year in his native Issyk-Kul Province about plans to develop Kyzyl-Ompol – a uranium deposit licensed to a private investor at the time the ban was enforced – Japarov said the site would now be “100%” developed by the state.
The lifting of the ban comes as Kyrgyzstan is considering the possibility of de